# NBA Team Trend Analysis

### Module Imports

In [15]:
from os import path
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
from datetime import date

### Global Variables and Settings

In [16]:
pd.options.display.max_columns = None

InteractiveShell.ast_node_interactivity = "all"

DATA_DIR = 'C:\\Users\\Harry\\Documents\\LTCWFF\\ltcwff_files\\data'

teams = ['ATL', 'BOS', 'BRK', 'CHO', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS']

pct_cols = ['Cover Pct', 'Over Pct', '1H Cover Pct', '1H Over Pct', '2H Cover Pct', '2H Over Pct', '1Q Cover Pct', '1Q Over Pct', '2Q Cover Pct', '2Q Over Pct', '3Q Cover Pct', '3Q Over Pct', '4Q Cover Pct', '4Q Over Pct']

spread_cols = ['Cover Pct', '1H Cover Pct', '2H Cover Pct', '1Q Cover Pct', '2Q Cover Pct', '3Q Cover Pct', '4Q Cover Pct']

total_cols = ['Over Pct', '1H Over Pct', '2H Over Pct', '1Q Over Pct', '2Q Over Pct', '3Q Over Pct', '4Q Over Pct']

### Load Data

In [17]:
def load_data():
    game_data = pd.read_csv(path.join(DATA_DIR, 'scraped_nba_combined_2021_compressed.csv'))
    game_data = game_data.set_index('Index')
    for col in game_data.columns:
        try:
            game_data[col] = game_data.to_numeric(df[col])
        except:
            continue
    return game_data

In [18]:
game_data = load_data()

latest_date = str(game_data.loc[game_data.index[len(game_data.index) - 1], 'Date'])

year = int(latest_date[0:4])
month = int(latest_date[4:6])
day = int(latest_date[6:8])

toall = date.today()
toyear = int(toall.strftime("%Y"))
tomonth = int(toall.strftime("%m"))
today = int(toall.strftime("%d"))

if toyear >= year and (tomonth > month or (tomonth == month and today > day + 1)):
    os.system('python update_nba_boxscores_compressed.py')
    os.system('python update_nba_odds_compressed.py')
    os.system('python combine_boxscore_odds_compressed.py')

    game_data = load_data()

### Analyze Team Trends

In [19]:
def get_team_data(team):
    home = game_data.loc[game_data['Home Team'] == team, :]

    home_cols = [ col.split('Home ')[1] if 'Home' in col else f"Opp {col.split('Away ')[1]}" if 'Away' in col else col for col in home.columns ]
    home.columns = home_cols

    away = game_data.loc[game_data['Away Team'] == team, :]
    away_cols = [ col.split('Away ')[1] if 'Away' in col else f"Opp {col.split('Home ')[1]}" if 'Home' in col else col for col in away.columns ]
    away.columns = away_cols

    return pd.concat([home, away])

In [20]:
def get_odds_data(team, recent = 0):
    #team = input('Team: ')

    combined = get_team_data(team)
    
    if recent > 0 and recent <= len(combined.index):
        combined = combined.tail(recent)

    team_odds = {}
    team_odds['Team'] = team 
    team_odds['Games'] = len(combined.index)
    team_odds['Average Spread'] = combined['Spread'].mean()
    team_odds['Median Margin'] = (combined['T'] - combined['Opp T']).median()
    team_odds['Covers'] = len(combined.loc[combined['T'] - combined['Opp T'] > -combined['Spread']].index)
    team_odds['Not Covers'] = len(combined.loc[combined['T'] - combined['Opp T'] < -combined['Spread']].index)
    team_odds['Pushes'] = len(combined.loc[combined['T'] - combined['Opp T'] == -combined['Spread']].index)
    team_odds['Cover Pct'] = team_odds['Covers'] / (team_odds['Covers'] + team_odds['Not Covers'] + team_odds['Pushes'])
    team_odds['Average Total'] = combined['Over'].mean()
    team_odds['Median Points'] = (combined['T'] + combined['Opp T']).median()
    team_odds['Overs'] = len(combined.loc[combined['T'] + combined['Opp T'] > combined['Over']].index)
    team_odds['Unders'] = len(combined.loc[combined['T'] + combined['Opp T'] < combined['Over']].index)
    team_odds['Total Pushes'] = len(combined.loc[combined['T'] + combined['Opp T'] == combined['Over']].index)
    team_odds['Over Pct'] = team_odds['Overs'] / (team_odds['Overs'] + team_odds['Unders'] + team_odds['Total Pushes'])
    team_odds['1H Average Spread'] = combined['1H Spread'].mean()
    team_odds['1H Median Margin'] = (combined['1'] + combined['2'] - combined['Opp 1'] - combined['Opp 2']).median()
    team_odds['1H Covers'] = len(combined.loc[combined['1'] + combined['2'] - combined['Opp 1'] - combined['Opp 2'] > -combined['1H Spread']].index)
    team_odds['1H Not Covers'] = len(combined.loc[combined['1'] + combined['2'] - combined['Opp 1'] - combined['Opp 2'] < -combined['1H Spread']].index)
    team_odds['1H Pushes'] = len(combined.loc[combined['1'] + combined['2'] - combined['Opp 1'] - combined['Opp 2'] == -combined['1H Spread']].index)
    team_odds['1H Cover Pct'] = team_odds['1H Covers'] / (team_odds['1H Covers'] + team_odds['1H Not Covers'] + team_odds['1H Pushes'])    
    team_odds['1H Average Total'] = combined['1H Over'].mean()
    team_odds['1H Median Points'] = (combined['1'] + combined['2'] + combined['Opp 1'] + combined['Opp 2']).median()
    team_odds['1H Overs'] = len(combined.loc[combined['1'] + combined['2'] + combined['Opp 1'] + combined['Opp 2'] > combined['1H Over']].index)
    team_odds['1H Unders'] = len(combined.loc[combined['1'] + combined['2'] + combined['Opp 1'] + combined['Opp 2'] < combined['1H Over']].index)
    team_odds['1H Total Pushes'] = len(combined.loc[combined['1'] + combined['2'] + combined['Opp 1'] + combined['Opp 2'] == combined['1H Over']].index)
    team_odds['1H Over Pct'] = team_odds['1H Overs'] / (team_odds['1H Overs'] + team_odds['1H Unders'] + team_odds['1H Total Pushes'])
    team_odds['2H Average Spread'] = combined['2H Spread'].mean()
    team_odds['2H Median Margin'] = (combined['3'] + combined['4'] - combined['Opp 3'] - combined['Opp 4']).median()
    team_odds['2H Covers'] = len(combined.loc[combined['3'] + combined['4'] - combined['Opp 3'] - combined['Opp 4'] > -combined['2H Spread']].index)
    team_odds['2H Not Covers'] = len(combined.loc[combined['3'] + combined['4'] - combined['Opp 3'] - combined['Opp 4'] < -combined['2H Spread']].index)
    team_odds['2H Pushes'] = len(combined.loc[combined['3'] + combined['4'] - combined['Opp 3'] - combined['Opp 4'] == -combined['2H Spread']].index)
    team_odds['2H Cover Pct'] = team_odds['2H Covers'] / (team_odds['2H Covers'] + team_odds['2H Not Covers'] + team_odds['2H Pushes'])
    team_odds['2H Average Total'] = combined['2H Over'].mean()
    team_odds['2H Median Points'] = (combined['3'] + combined['4'] + combined['Opp 3'] + combined['Opp 4']).median()
    team_odds['2H Overs'] = len(combined.loc[combined['3'] + combined['4'] + combined['Opp 3'] + combined['Opp 4'] > combined['2H Over']].index)
    team_odds['2H Unders'] = len(combined.loc[combined['3'] + combined['4'] + combined['Opp 3'] + combined['Opp 4'] < combined['2H Over']].index)
    team_odds['2H Total Pushes'] = len(combined.loc[combined['3'] + combined['4'] + combined['Opp 3'] + combined['Opp 4'] == combined['2H Over']].index)
    team_odds['2H Over Pct'] = team_odds['2H Overs'] / (team_odds['2H Overs'] + team_odds['2H Unders'] + team_odds['2H Total Pushes'])
    team_odds['1Q Average Spread'] = combined['1Q Spread'].mean()
    team_odds['1Q Median Margin'] = (combined['1'] - combined['Opp 1']).median()
    team_odds['1Q Covers'] = len(combined.loc[combined['1'] - combined['Opp 1'] > -combined['1Q Spread']].index)
    team_odds['1Q Not Covers'] = len(combined.loc[combined['1'] - combined['Opp 1'] < -combined['1Q Spread']].index)
    team_odds['1Q Pushes'] = len(combined.loc[combined['1'] - combined['Opp 1'] == -combined['1Q Spread']].index)
    team_odds['1Q Cover Pct'] = team_odds['1Q Covers'] / (team_odds['1Q Covers'] + team_odds['1Q Not Covers'] + team_odds['1Q Pushes'])
    team_odds['1Q Average Total'] = combined['1Q Over'].mean()
    team_odds['1Q Median Points'] = (combined['1'] + combined['Opp 1']).median()
    team_odds['1Q Overs'] = len(combined.loc[combined['1'] + combined['Opp 1'] > combined['1Q Over']].index)
    team_odds['1Q Unders'] = len(combined.loc[combined['1'] + combined['Opp 1'] < combined['1Q Over']].index)
    team_odds['1Q Total Pushes'] = len(combined.loc[combined['1'] + combined['Opp 1'] == combined['1Q Over']].index)
    team_odds['1Q Over Pct'] = team_odds['1Q Overs'] / (team_odds['1Q Overs'] + team_odds['1Q Unders'] + team_odds['1Q Total Pushes'])
    team_odds['2Q Average Spread'] = combined['2Q Spread'].mean()
    team_odds['2Q Median Margin'] = (combined['2'] - combined['Opp 2']).median()
    team_odds['2Q Covers'] = len(combined.loc[combined['2'] - combined['Opp 2'] > -combined['2Q Spread']].index)
    team_odds['2Q Not Covers'] = len(combined.loc[combined['2'] - combined['Opp 2'] < -combined['2Q Spread']].index)
    team_odds['2Q Pushes'] = len(combined.loc[combined['2'] - combined['Opp 2'] == -combined['2Q Spread']].index)
    team_odds['2Q Cover Pct'] = team_odds['2Q Covers'] / (team_odds['2Q Covers'] + team_odds['2Q Not Covers'] + team_odds['2Q Pushes'])
    team_odds['2Q Average Total'] = combined['2Q Over'].mean()
    team_odds['2Q Median Points'] = (combined['2'] + combined['Opp 2']).median()
    team_odds['2Q Overs'] = len(combined.loc[combined['2'] + combined['Opp 2'] > combined['2Q Over']].index)
    team_odds['2Q Unders'] = len(combined.loc[combined['2'] + combined['Opp 2'] < combined['2Q Over']].index)
    team_odds['2Q Total Pushes'] = len(combined.loc[combined['2'] + combined['Opp 2'] == combined['2Q Over']].index)
    team_odds['2Q Over Pct'] = team_odds['2Q Overs'] / (team_odds['2Q Overs'] + team_odds['2Q Unders'] + team_odds['2Q Total Pushes'])
    team_odds['3Q Average Spread'] = combined['3Q Spread'].mean()
    team_odds['3Q Median Margin'] = (combined['3'] - combined['Opp 3']).median()
    team_odds['3Q Covers'] = len(combined.loc[combined['3'] - combined['Opp 3'] > -combined['3Q Spread']].index)
    team_odds['3Q Not Covers'] = len(combined.loc[combined['3'] - combined['Opp 3'] < -combined['3Q Spread']].index)
    team_odds['3Q Pushes'] = len(combined.loc[combined['3'] - combined['Opp 3'] == -combined['3Q Spread']].index)
    team_odds['3Q Cover Pct'] = team_odds['3Q Covers'] / (team_odds['3Q Covers'] + team_odds['3Q Not Covers'] + team_odds['3Q Pushes'])
    team_odds['3Q Average Total'] = combined['3Q Over'].mean()
    team_odds['3Q Median Points'] = (combined['3'] + combined['Opp 3']).median()
    team_odds['3Q Overs'] = len(combined.loc[combined['3'] + combined['Opp 3'] > combined['3Q Over']].index)
    team_odds['3Q Unders'] = len(combined.loc[combined['3'] + combined['Opp 3'] < combined['3Q Over']].index)
    team_odds['3Q Total Pushes'] = len(combined.loc[combined['3'] + combined['Opp 3'] == combined['3Q Over']].index)
    team_odds['3Q Over Pct'] = team_odds['3Q Overs'] / (team_odds['3Q Overs'] + team_odds['3Q Unders'] + team_odds['3Q Total Pushes'])
    team_odds['4Q Average Spread'] = combined['4Q Spread'].mean()
    team_odds['4Q Median Margin'] = (combined['4'] - combined['Opp 4']).median()
    team_odds['4Q Covers'] = len(combined.loc[combined['4'] - combined['Opp 4'] > -combined['4Q Spread']].index)
    team_odds['4Q Not Covers'] = len(combined.loc[combined['4'] - combined['Opp 4'] < -combined['4Q Spread']].index)
    team_odds['4Q Pushes'] = len(combined.loc[combined['4'] - combined['Opp 4'] == -combined['4Q Spread']].index)
    team_odds['4Q Cover Pct'] = team_odds['4Q Covers'] / (team_odds['4Q Covers'] + team_odds['4Q Not Covers'] + team_odds['4Q Pushes'])
    team_odds['4Q Average Total'] = combined['4Q Over'].mean()
    team_odds['4Q Median Points'] = (combined['4'] + combined['Opp 4']).median()
    team_odds['4Q Overs'] = len(combined.loc[combined['4'] + combined['Opp 4'] > combined['4Q Over']].index)
    team_odds['4Q Unders'] = len(combined.loc[combined['4'] + combined['Opp 4'] < combined['4Q Over']].index)
    team_odds['4Q Total Pushes'] = len(combined.loc[combined['4'] + combined['Opp 4'] == combined['4Q Over']].index)
    team_odds['4Q Over Pct'] = team_odds['4Q Overs'] / (team_odds['4Q Overs'] + team_odds['4Q Unders'] + team_odds['4Q Total Pushes'])

    pd.DataFrame(team_odds, index = [0])

    return team_odds

In [21]:
rows = []

for team in teams:
    rows.append(get_odds_data(team))

df = pd.DataFrame(rows)

df.style.apply(lambda x: ["background: green" if type(v) != str and (v > .65 or v < .35) else "" for v in x], axis = 1, subset = pct_cols)

,Team,Games,Average Spread,Median Margin,Covers,Not Covers,Pushes,Cover Pct,Average Total,Median Points,Overs,Unders,Total Pushes,Over Pct,1H Average Spread,1H Median Margin,1H Covers,1H Not Covers,1H Pushes,1H Cover Pct,1H Average Total,1H Median Points,1H Overs,1H Unders,1H Total Pushes,1H Over Pct,2H Average Spread,2H Median Margin,2H Covers,2H Not Covers,2H Pushes,2H Cover Pct,2H Average Total,2H Median Points,2H Overs,2H Unders,2H Total Pushes,2H Over Pct,1Q Average Spread,1Q Median Margin,1Q Covers,1Q Not Covers,1Q Pushes,1Q Cover Pct,1Q Average Total,1Q Median Points,1Q Overs,1Q Unders,1Q Total Pushes,1Q Over Pct,2Q Average Spread,2Q Median Margin,2Q Covers,2Q Not Covers,2Q Pushes,2Q Cover Pct,2Q Average Total,2Q Median Points,2Q Overs,2Q Unders,2Q Total Pushes,2Q Over Pct,3Q Average Spread,3Q Median Margin,3Q Covers,3Q Not Covers,3Q Pushes,3Q Cover Pct,3Q Average Total,3Q Median Points,3Q Overs,3Q Unders,3Q Total Pushes,3Q Over Pct,4Q Average Spread,4Q Median Margin,4Q Covers,4Q Not Covers,4Q Pushes,4Q Cover Pct,4Q Average Total,4Q Median Points,4Q Overs,4Q Unders,4Q Total Pushes,4Q Over Pct
0,ATL,34,-0.132353,-4.000000,16,18,0,0.470588,227.852941,227.500000,15,18,1,0.441176,-0.132353,1.500000,18,15,1,0.529412,115.308824,113.000000,14,20,0,0.411765,0.161765,0.000000,17,17,0,0.500000,112.632353,112.000000,18,16,0,0.529412,-0.014706,-1.000000,14,20,0,0.411765,57.720588,56.500000,16,18,0,0.470588,-0.014706,0.000000,18,15,1,0.529412,57.176471,55.500000,13,20,1,0.382353,0.058824,2.500000,20,14,0,0.588235,56.382353,55.500000,14,19,1,0.411765,0.000000,-4.000000,11,23,0,0.323529,55.838235,56.000000,18,15,1,0.529412
1,BOS,34,-1.485294,0.000000,16,18,0,0.470588,220.911765,224.500000,18,16,0,0.529412,-0.808824,3.000000,19,13,2,0.558824,109.647059,108.000000,13,19,2,0.382353,0.279412,-3.500000,12,21,1,0.352941,111.676471,112.000000,15,17,2,0.441176,-0.558824,2.000000,18,15,1,0.529412,55.029412,57.000000,20,12,2,0.588235,-0.424242,2.000000,20,12,1,0.606061,54.484848,54.000000,15,18,0,0.454545,-0.014706,-1.000000,14,19,1,0.411765,55.779412,58.000000,20,13,1,0.588235,-0.287879,-1.000000,14,17,2,0.424242,55.075758,55.000000,19,14,0,0.575758
2,BRK,36,-2.902778,4.000000,19,17,0,0.527778,232.819444,237.000000,24,12,0,0.666667,-1.666667,1.500000,19,17,0,0.527778,117.902778,115.000000,15,19,2,0.416667,0.542857,0.000000,20,15,0,0.571429,115.214286,116.000000,19,15,1,0.542857,-1.111111,-1.000000,14,21,1,0.388889,59.138889,57.000000,16,20,0,0.444444,-0.736111,2.500000,20,13,3,0.555556,58.500000,57.000000,15,21,0,0.416667,-0.680556,0.500000,15,20,1,0.416667,57.722222,60.500000,23,12,1,0.638889,-0.541667,0.500000,18,17,1,0.500000,56.791667,56.500000,16,19,1,0.444444
3,CHO,34,4.250000,-2.500000,18,15,1,0.529412,224.294118,226.500000,20,14,0,0.588235,2.426471,-2.000000,16,17,1,0.470588,113.588235,116.500000,18,14,2,0.529412,0.338235,2.500000,21,13,0,0.617647,110.882353,109.000000,14,20,0,0.411765,1.485294,-1.000000,19,14,1,0.558824,56.882353,57.500000,19,15,0,0.558824,0.897059,-1.500000,13,20,1,0.382353,56.294118,60.000000,23,10,1,0.676471,0.794118,0.500000,20,13,1,0.588235,55.411765,54.000000,13,20,1,0.382353,0.632353,0.000000,19,14,1,0.558824,54.647059,55.500000,18,16,0,0.529412
4,CHI,33,3.424242,-1.000000,18,14,1,0.545455,227.227273,230.000000,18,15,0,0.545455,1.969697,-3.000000,17,15,1,0.515152,113.681818,115.000000,18,15,0,0.545455,-0.015625,1.000000,20,11,1,0.625000,113.734375,113.000000,17,15,0,0.531250,1.196970,0.000000,19,14,0,0.575758,57.060606,57.000000,16,17,0,0.484848,0.757576,0.000000,17,15,1,0.515152,56.545455,56.000000,16,15,2,0.484848,0.787879,2.000000,20,13,0,0.606061,56.757576,56.000000,15,18,0,0.454545,0.636364,-2.000000,14,17,2,0.424242,56.121212,54.000000,13,20,0,0.393939
5,CLE,35,5.557143,-9.000000,14,21,0,0.400000,218.000000,220.000000,13,21,1,0.371429,3.114286,-3.000000,17,16,2,0.485714,110.285714,109.000000,15,19,1,0.428571,0.397059,-5.000000,12,20,2,0.352941,107.367647,105.000000,16,18,0,0.470588,1.828571,-2.000000,18,1

In [22]:
def compare_teams(team_1, team_2, recent = 0):
    matchup_rows = []

    matchup_rows.append(get_odds_data(team_1, recent))
    matchup_rows.append(get_odds_data(team_2, recent))

    return pd.DataFrame(matchup_rows)

In [23]:
rows = []

for team in teams:
    rows.append(get_odds_data(team))

df = pd.DataFrame(rows)

df.style.apply(lambda x: ["background: green" if type(v) != str and (v > .65 or v < .35) else "" for v in x], axis = 1, subset = pct_cols)

,Team,Games,Average Spread,Median Margin,Covers,Not Covers,Pushes,Cover Pct,Average Total,Median Points,Overs,Unders,Total Pushes,Over Pct,1H Average Spread,1H Median Margin,1H Covers,1H Not Covers,1H Pushes,1H Cover Pct,1H Average Total,1H Median Points,1H Overs,1H Unders,1H Total Pushes,1H Over Pct,2H Average Spread,2H Median Margin,2H Covers,2H Not Covers,2H Pushes,2H Cover Pct,2H Average Total,2H Median Points,2H Overs,2H Unders,2H Total Pushes,2H Over Pct,1Q Average Spread,1Q Median Margin,1Q Covers,1Q Not Covers,1Q Pushes,1Q Cover Pct,1Q Average Total,1Q Median Points,1Q Overs,1Q Unders,1Q Total Pushes,1Q Over Pct,2Q Average Spread,2Q Median Margin,2Q Covers,2Q Not Covers,2Q Pushes,2Q Cover Pct,2Q Average Total,2Q Median Points,2Q Overs,2Q Unders,2Q Total Pushes,2Q Over Pct,3Q Average Spread,3Q Median Margin,3Q Covers,3Q Not Covers,3Q Pushes,3Q Cover Pct,3Q Average Total,3Q Median Points,3Q Overs,3Q Unders,3Q Total Pushes,3Q Over Pct,4Q Average Spread,4Q Median Margin,4Q Covers,4Q Not Covers,4Q Pushes,4Q Cover Pct,4Q Average Total,4Q Median Points,4Q Overs,4Q Unders,4Q Total Pushes,4Q Over Pct
0,ATL,34,-0.132353,-4.000000,16,18,0,0.470588,227.852941,227.500000,15,18,1,0.441176,-0.132353,1.500000,18,15,1,0.529412,115.308824,113.000000,14,20,0,0.411765,0.161765,0.000000,17,17,0,0.500000,112.632353,112.000000,18,16,0,0.529412,-0.014706,-1.000000,14,20,0,0.411765,57.720588,56.500000,16,18,0,0.470588,-0.014706,0.000000,18,15,1,0.529412,57.176471,55.500000,13,20,1,0.382353,0.058824,2.500000,20,14,0,0.588235,56.382353,55.500000,14,19,1,0.411765,0.000000,-4.000000,11,23,0,0.323529,55.838235,56.000000,18,15,1,0.529412
1,BOS,34,-1.485294,0.000000,16,18,0,0.470588,220.911765,224.500000,18,16,0,0.529412,-0.808824,3.000000,19,13,2,0.558824,109.647059,108.000000,13,19,2,0.382353,0.279412,-3.500000,12,21,1,0.352941,111.676471,112.000000,15,17,2,0.441176,-0.558824,2.000000,18,15,1,0.529412,55.029412,57.000000,20,12,2,0.588235,-0.424242,2.000000,20,12,1,0.606061,54.484848,54.000000,15,18,0,0.454545,-0.014706,-1.000000,14,19,1,0.411765,55.779412,58.000000,20,13,1,0.588235,-0.287879,-1.000000,14,17,2,0.424242,55.075758,55.000000,19,14,0,0.575758
2,BRK,36,-2.902778,4.000000,19,17,0,0.527778,232.819444,237.000000,24,12,0,0.666667,-1.666667,1.500000,19,17,0,0.527778,117.902778,115.000000,15,19,2,0.416667,0.542857,0.000000,20,15,0,0.571429,115.214286,116.000000,19,15,1,0.542857,-1.111111,-1.000000,14,21,1,0.388889,59.138889,57.000000,16,20,0,0.444444,-0.736111,2.500000,20,13,3,0.555556,58.500000,57.000000,15,21,0,0.416667,-0.680556,0.500000,15,20,1,0.416667,57.722222,60.500000,23,12,1,0.638889,-0.541667,0.500000,18,17,1,0.500000,56.791667,56.500000,16,19,1,0.444444
3,CHO,34,4.250000,-2.500000,18,15,1,0.529412,224.294118,226.500000,20,14,0,0.588235,2.426471,-2.000000,16,17,1,0.470588,113.588235,116.500000,18,14,2,0.529412,0.338235,2.500000,21,13,0,0.617647,110.882353,109.000000,14,20,0,0.411765,1.485294,-1.000000,19,14,1,0.558824,56.882353,57.500000,19,15,0,0.558824,0.897059,-1.500000,13,20,1,0.382353,56.294118,60.000000,23,10,1,0.676471,0.794118,0.500000,20,13,1,0.588235,55.411765,54.000000,13,20,1,0.382353,0.632353,0.000000,19,14,1,0.558824,54.647059,55.500000,18,16,0,0.529412
4,CHI,33,3.424242,-1.000000,18,14,1,0.545455,227.227273,230.000000,18,15,0,0.545455,1.969697,-3.000000,17,15,1,0.515152,113.681818,115.000000,18,15,0,0.545455,-0.015625,1.000000,20,11,1,0.625000,113.734375,113.000000,17,15,0,0.531250,1.196970,0.000000,19,14,0,0.575758,57.060606,57.000000,16,17,0,0.484848,0.757576,0.000000,17,15,1,0.515152,56.545455,56.000000,16,15,2,0.484848,0.787879,2.000000,20,13,0,0.606061,56.757576,56.000000,15,18,0,0.454545,0.636364,-2.000000,14,17,2,0.424242,56.121212,54.000000,13,20,0,0.393939
5,CLE,35,5.557143,-9.000000,14,21,0,0.400000,218.000000,220.000000,13,21,1,0.371429,3.114286,-3.000000,17,16,2,0.485714,110.285714,109.000000,15,19,1,0.428571,0.397059,-5.000000,12,20,2,0.352941,107.367647,105.000000,16,18,0,0.470588,1.828571,-2.000000,18,1

In [24]:
extremes = df.loc[((df[pct_cols] > 0.7) | (df[pct_cols] < 0.3)).any(axis = 1)]

extremes.style.apply(lambda x: ["background: green" if type(v) != str and v > .7 else "background:red" if type(v) != str and v < .3 else "" for v in x], axis = 1, subset = pct_cols)

,Team,Games,Average Spread,Median Margin,Covers,Not Covers,Pushes,Cover Pct,Average Total,Median Points,Overs,Unders,Total Pushes,Over Pct,1H Average Spread,1H Median Margin,1H Covers,1H Not Covers,1H Pushes,1H Cover Pct,1H Average Total,1H Median Points,1H Overs,1H Unders,1H Total Pushes,1H Over Pct,2H Average Spread,2H Median Margin,2H Covers,2H Not Covers,2H Pushes,2H Cover Pct,2H Average Total,2H Median Points,2H Overs,2H Unders,2H Total Pushes,2H Over Pct,1Q Average Spread,1Q Median Margin,1Q Covers,1Q Not Covers,1Q Pushes,1Q Cover Pct,1Q Average Total,1Q Median Points,1Q Overs,1Q Unders,1Q Total Pushes,1Q Over Pct,2Q Average Spread,2Q Median Margin,2Q Covers,2Q Not Covers,2Q Pushes,2Q Cover Pct,2Q Average Total,2Q Median Points,2Q Overs,2Q Unders,2Q Total Pushes,2Q Over Pct,3Q Average Spread,3Q Median Margin,3Q Covers,3Q Not Covers,3Q Pushes,3Q Cover Pct,3Q Average Total,3Q Median Points,3Q Overs,3Q Unders,3Q Total Pushes,3Q Over Pct,4Q Average Spread,4Q Median Margin,4Q Covers,4Q Not Covers,4Q Pushes,4Q Cover Pct,4Q Average Total,4Q Median Points,4Q Overs,4Q Unders,4Q Total Pushes,4Q Over Pct
7,DEN,34,-3.764706,4.000000,15,19,0,0.441176,223.573529,228.500000,22,11,1,0.647059,-2.044118,7.000000,20,14,0,0.588235,113.426471,116.500000,21,13,0,0.617647,1.227273,2.000000,16,14,3,0.484848,110.000000,106.000000,12,21,0,0.363636,-1.250000,5.500000,21,12,1,0.617647,56.911765,58.500000,19,13,2,0.558824,-0.779412,1.000000,17,17,0,0.500000,56.220588,56.500000,17,16,1,0.500000,-0.573529,-1.500000,10,23,1,0.294118,55.235294,56.500000,20,11,3,0.588235,-0.617647,1.500000,18,16,0,0.529412,54.632353,52.000000,11,23,0,0.323529
13,LAL,35,-6.400000,7.000000,16,19,0,0.457143,221.814286,218.000000,13,22,0,0.371429,-3.585714,1.000000,13,20,2,0.371429,112.914286,112.000000,14,20,1,0.400000,-2.191176,1.000000,17,17,0,0.500000,109.485294,100.000000,8,26,0,0.235294,-2.100000,-2.000000,16,19,0,0.457143,56.500000,57.000000,19,16,0,0.542857,-1.328571,4.000000,24,11,0,0.685714,55.914286,56.000000,17,17,1,0.485714,-1.971429,0.000000,14,20,1,0.400000,54.842857,52.000000,13,22,0,0.371429,-1.000000,4.000000,23,12,0,0.657143,53.857143,49.000000,9,26,0,0.257143
18,NOP,34,-0.220588,-2.000000,15,19,0,0.441176,225.220588,231.000000,24,9,1,0.705882,-0.176471,0.000000,17,17,0,0.500000,113.926471,117.000000,20,13,1,0.588235,-0.117647,-2.000000,15,18,1,0.441176,111.705882,111.000000,18,16,0,0.529412,-0.147059,2.000000,18,15,1,0.529412,57.073529,59.500000,22,12,0,0.647059,0.090909,-1.000000,12,19,2,0.363636,56.424242,58.000000,21,10,2,0.636364,-0.073529,0.000000,16,17,1,0.470588,55.926471,57.500000,22,12,0,0.647059,-0.015152,1.500000,18,15,0,0.545455,55.030303,55.000000,19,14,0,0.575758
19,NYK,35,3.528571,3.000000,20,15,0,0.571429,214.257143,209.000000,12,23,0,0.342857,1.985714,3.000000,25,9,1,0.714286,108.400000,108.000000,14,19,2,0.400000,2.214286,0.000000,18,17,0,0.514286,105.771429,103.000000,14,21,0,0.400000,1.214286,2.000000,22,12,1,0.628571,54.257143,53.000000,15,20,0,0.428571,0.742857,1.000000,20,14,1,0.571429,53.900000,53.000000,17,17,1,0.485714,1.571429,1.000000,21,14,0,0.600000,52.985714,49.000000,11,24,0,0.314286,0.557143,-1.000000,16,16,3,0.457143,52.171429,52.000000,19,16,0,0.542857
27,TOR,34,-1.367647,1.000000,16,18,0,0.470588,223.426471,225.000000,19,15,0,0.558824,-0.720588,1.000000,15,18,1,0.441176,113.382353,115.000000,19,14,1,0.558824,-0.242424,1.500000,15,18,0,0.454545,109.969697,107.500000,14,19,0,0.424242,-0.441176,3.000000,22,11,1,0.647059,56.808824,57.500000,18,16,0,0.529412,-0.264706,-3.000000,14,19,1,0.411765,56.220588,59.000000,24,10,0,0.705882,-0.250000,-2.000000,16,18,0,0.470588,55.161765,55.000000,17,15,2,0.500000,-0.176471,2.000000,21,12,1,0.617647,54.382353,54.000000,16,18,0,0.470588
28,UTA,35,-6.071429,13.000000,25,10,0,0.714286,222.442857,221.000000,17,17,1,0.485714,-3.314286,6.000000,20,15,0,0.571429,110.785714,107.000000,15,20,0,0.428571,0.928571,6.000000,24,10,1,0.685714,111.842857,113.000000,20,15,0,0.571429,-1.971429,1.000000,

In [25]:
team_1 = input('Team 1: ')
team_2 = input('Team 2: ')


matchup_df = compare_teams(team_1, team_2)
matchup_df.index.name = 'All'
matchup_df.style.apply(lambda x: ["background: green" if type(v) != str and (v > .7 or v < .3) else "" for v in x], axis = 1, subset = pct_cols)

matchup_df.index.name = 'Spreads'
matchup_df.style.apply(lambda y: ["background: blue", "background: blue"] if (y[0] > .6 and y[1] < .4) or (y[0] < .4 and y[1] > .6) else ["", ""], axis = 0, subset = spread_cols)

matchup_df.index.name = 'Totals'
matchup_df.style.apply(lambda y: ["background: blue", "background: blue"] if (y[0] > .6 and y[1] > .6) or (y[0] < .4 and y[1] < .4) else ["", ""], axis = 0, subset = total_cols)

recent_df = compare_teams(team_1, team_2, 8)
recent_df.index.name = 'Recent'
recent_df.style.apply(lambda x: ["background: green" if type(v) != str and (v > .7 or v < .3) else "" for v in x], axis = 1, subset = pct_cols)

,Team,Games,Average Spread,Median Margin,Covers,Not Covers,Pushes,Cover Pct,Average Total,Median Points,Overs,Unders,Total Pushes,Over Pct,1H Average Spread,1H Median Margin,1H Covers,1H Not Covers,1H Pushes,1H Cover Pct,1H Average Total,1H Median Points,1H Overs,1H Unders,1H Total Pushes,1H Over Pct,2H Average Spread,2H Median Margin,2H Covers,2H Not Covers,2H Pushes,2H Cover Pct,2H Average Total,2H Median Points,2H Overs,2H Unders,2H Total Pushes,2H Over Pct,1Q Average Spread,1Q Median Margin,1Q Covers,1Q Not Covers,1Q Pushes,1Q Cover Pct,1Q Average Total,1Q Median Points,1Q Overs,1Q Unders,1Q Total Pushes,1Q Over Pct,2Q Average Spread,2Q Median Margin,2Q Covers,2Q Not Covers,2Q Pushes,2Q Cover Pct,2Q Average Total,2Q Median Points,2Q Overs,2Q Unders,2Q Total Pushes,2Q Over Pct,3Q Average Spread,3Q Median Margin,3Q Covers,3Q Not Covers,3Q Pushes,3Q Cover Pct,3Q Average Total,3Q Median Points,3Q Overs,3Q Unders,3Q Total Pushes,3Q Over Pct,4Q Average Spread,4Q Median Margin,4Q Covers,4Q Not Covers,4Q Pushes,4Q Cover Pct,4Q Average Total,4Q Median Points,4Q Overs,4Q Unders,4Q Total Pushes,4Q Over Pct
All,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,BOS,34,-1.485294,0.000000,16,18,0,0.470588,220.911765,224.500000,18,16,0,0.529412,-0.808824,3.000000,19,13,2,0.558824,109.647059,108.000000,13,19,2,0.382353,0.279412,-3.500000,12,21,1,0.352941,111.676471,112.000000,15,17,2,0.441176,-0.558824,2.000000,18,15,1,0.529412,55.029412,57.000000,20,12,2,0.588235,-0.424242,2.000000,20,12,1,0.606061,54.484848,54.000000,15,18,0,0.454545,-0.014706,-1.000000,14,19,1,0.411765,55.779412,58.000000,20,13,1,0.588235,-0.287879,-1.000000,14,17,2,0.424242,55.075758,55.000000,19,14,0,0.575758
1,LAC,36,-5.541667,7.000000,20,16,0,0.555556,224.166667,224.000000,19,15,2,0.527778,-3.027778,5.000000,21,14,1,0.583333,113.666667,110.000000,12,24,0,0.333333,-0.416667,3.000000,20,16,0,0.555556,110.652778,112.000000,19,16,1,0.527778,-1.819444,3.000000,20,13,3,0.555556,56.986111,55.500000,13,21,2,0.361111,-1.250000,1.500000,18,18,0,0.500000,56.486111,57.500000,17,18,1,0.472222,-0.861111,3.000000,21,15,0,0.583333,55.527778,57.000000,20,14,2,0.555556,-0.847222,-0.500000,15,20,1,0.416667,54.652778,55.500000,19,16,1,0.527778


,Team,Games,Average Spread,Median Margin,Covers,Not Covers,Pushes,Cover Pct,Average Total,Median Points,Overs,Unders,Total Pushes,Over Pct,1H Average Spread,1H Median Margin,1H Covers,1H Not Covers,1H Pushes,1H Cover Pct,1H Average Total,1H Median Points,1H Overs,1H Unders,1H Total Pushes,1H Over Pct,2H Average Spread,2H Median Margin,2H Covers,2H Not Covers,2H Pushes,2H Cover Pct,2H Average Total,2H Median Points,2H Overs,2H Unders,2H Total Pushes,2H Over Pct,1Q Average Spread,1Q Median Margin,1Q Covers,1Q Not Covers,1Q Pushes,1Q Cover Pct,1Q Average Total,1Q Median Points,1Q Overs,1Q Unders,1Q Total Pushes,1Q Over Pct,2Q Average Spread,2Q Median Margin,2Q Covers,2Q Not Covers,2Q Pushes,2Q Cover Pct,2Q Average Total,2Q Median Points,2Q Overs,2Q Unders,2Q Total Pushes,2Q Over Pct,3Q Average Spread,3Q Median Margin,3Q Covers,3Q Not Covers,3Q Pushes,3Q Cover Pct,3Q Average Total,3Q Median Points,3Q Overs,3Q Unders,3Q Total Pushes,3Q Over Pct,4Q Average Spread,4Q Median Margin,4Q Covers,4Q Not Covers,4Q Pushes,4Q Cover Pct,4Q Average Total,4Q Median Points,4Q Overs,4Q Unders,4Q Total Pushes,4Q Over Pct
Spreads,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,BOS,34,-1.485294,0.000000,16,18,0,0.470588,220.911765,224.500000,18,16,0,0.529412,-0.808824,3.000000,19,13,2,0.558824,109.647059,108.000000,13,19,2,0.382353,0.279412,-3.500000,12,21,1,0.352941,111.676471,112.000000,15,17,2,0.441176,-0.558824,2.000000,18,15,1,0.529412,55.029412,57.000000,20,12,2,0.588235,-0.424242,2.000000,20,12,1,0.606061,54.484848,54.000000,15,18,0,0.454545,-0.014706,-1.000000,14,19,1,0.411765,55.779412,58.000000,20,13,1,0.588235,-0.287879,-1.000000,14,17,2,0.424242,55.075758,55.000000,19,14,0,0.575758
1,LAC,36,-5.541667,7.000000,20,16,0,0.555556,224.166667,224.000000,19,15,2,0.527778,-3.027778,5.000000,21,14,1,0.583333,113.666667,110.000000,12,24,0,0.333333,-0.416667,3.000000,20,16,0,0.555556,110.652778,112.000000,19,16,1,0.527778,-1.819444,3.000000,20,13,3,0.555556,56.986111,55.500000,13,21,2,0.361111,-1.250000,1.500000,18,18,0,0.500000,56.486111,57.500000,17,18,1,0.472222,-0.861111,3.000000,21,15,0,0.583333,55.527778,57.000000,20,14,2,0.555556,-0.847222,-0.500000,15,20,1,0.416667,54.652778,55.500000,19,16,1,0.527778


,Team,Games,Average Spread,Median Margin,Covers,Not Covers,Pushes,Cover Pct,Average Total,Median Points,Overs,Unders,Total Pushes,Over Pct,1H Average Spread,1H Median Margin,1H Covers,1H Not Covers,1H Pushes,1H Cover Pct,1H Average Total,1H Median Points,1H Overs,1H Unders,1H Total Pushes,1H Over Pct,2H Average Spread,2H Median Margin,2H Covers,2H Not Covers,2H Pushes,2H Cover Pct,2H Average Total,2H Median Points,2H Overs,2H Unders,2H Total Pushes,2H Over Pct,1Q Average Spread,1Q Median Margin,1Q Covers,1Q Not Covers,1Q Pushes,1Q Cover Pct,1Q Average Total,1Q Median Points,1Q Overs,1Q Unders,1Q Total Pushes,1Q Over Pct,2Q Average Spread,2Q Median Margin,2Q Covers,2Q Not Covers,2Q Pushes,2Q Cover Pct,2Q Average Total,2Q Median Points,2Q Overs,2Q Unders,2Q Total Pushes,2Q Over Pct,3Q Average Spread,3Q Median Margin,3Q Covers,3Q Not Covers,3Q Pushes,3Q Cover Pct,3Q Average Total,3Q Median Points,3Q Overs,3Q Unders,3Q Total Pushes,3Q Over Pct,4Q Average Spread,4Q Median Margin,4Q Covers,4Q Not Covers,4Q Pushes,4Q Cover Pct,4Q Average Total,4Q Median Points,4Q Overs,4Q Unders,4Q Total Pushes,4Q Over Pct
Totals,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,BOS,34,-1.485294,0.000000,16,18,0,0.470588,220.911765,224.500000,18,16,0,0.529412,-0.808824,3.000000,19,13,2,0.558824,109.647059,108.000000,13,19,2,0.382353,0.279412,-3.500000,12,21,1,0.352941,111.676471,112.000000,15,17,2,0.441176,-0.558824,2.000000,18,15,1,0.529412,55.029412,57.000000,20,12,2,0.588235,-0.424242,2.000000,20,12,1,0.606061,54.484848,54.000000,15,18,0,0.454545,-0.014706,-1.000000,14,19,1,0.411765,55.779412,58.000000,20,13,1,0.588235,-0.287879,-1.000000,14,17,2,0.424242,55.075758,55.000000,19,14,0,0.575758
1,LAC,36,-5.541667,7.000000,20,16,0,0.555556,224.166667,224.000000,19,15,2,0.527778,-3.027778,5.000000,21,14,1,0.583333,113.666667,110.000000,12,24,0,0.333333,-0.416667,3.000000,20,16,0,0.555556,110.652778,112.000000,19,16,1,0.527778,-1.819444,3.000000,20,13,3,0.555556,56.986111,55.500000,13,21,2,0.361111,-1.250000,1.500000,18,18,0,0.500000,56.486111,57.500000,17,18,1,0.472222,-0.861111,3.000000,21,15,0,0.583333,55.527778,57.000000,20,14,2,0.555556,-0.847222,-0.500000,15,20,1,0.416667,54.652778,55.500000,19,16,1,0.527778


,Team,Games,Average Spread,Median Margin,Covers,Not Covers,Pushes,Cover Pct,Average Total,Median Points,Overs,Unders,Total Pushes,Over Pct,1H Average Spread,1H Median Margin,1H Covers,1H Not Covers,1H Pushes,1H Cover Pct,1H Average Total,1H Median Points,1H Overs,1H Unders,1H Total Pushes,1H Over Pct,2H Average Spread,2H Median Margin,2H Covers,2H Not Covers,2H Pushes,2H Cover Pct,2H Average Total,2H Median Points,2H Overs,2H Unders,2H Total Pushes,2H Over Pct,1Q Average Spread,1Q Median Margin,1Q Covers,1Q Not Covers,1Q Pushes,1Q Cover Pct,1Q Average Total,1Q Median Points,1Q Overs,1Q Unders,1Q Total Pushes,1Q Over Pct,2Q Average Spread,2Q Median Margin,2Q Covers,2Q Not Covers,2Q Pushes,2Q Cover Pct,2Q Average Total,2Q Median Points,2Q Overs,2Q Unders,2Q Total Pushes,2Q Over Pct,3Q Average Spread,3Q Median Margin,3Q Covers,3Q Not Covers,3Q Pushes,3Q Cover Pct,3Q Average Total,3Q Median Points,3Q Overs,3Q Unders,3Q Total Pushes,3Q Over Pct,4Q Average Spread,4Q Median Margin,4Q Covers,4Q Not Covers,4Q Pushes,4Q Cover Pct,4Q Average Total,4Q Median Points,4Q Overs,4Q Unders,4Q Total Pushes,4Q Over Pct
Recent,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,BOS,8,-2.500000,3.500000,5,3,0,0.625000,220.500000,223.500000,4,4,0,0.500000,-1.250000,3.000000,4,3,1,0.500000,112.062500,114.000000,4,4,0,0.500000,-0.500000,0.000000,4,4,0,0.500000,108.375000,105.000000,3,3,2,0.375000,-1.125000,3.000000,5,2,1,0.625000,56.062500,58.500000,4,3,1,0.500000,-0.750000,1.000000,4,3,1,0.500000,55.625000,55.000000,4,4,0,0.500000,-0.312500,-1.000000,4,4,0,0.500000,54.500000,48.000000,2,6,0,0.250000,-0.500000,-0.500000,4,4,0,0.500000,54.125000,57.000000,6,2,0,0.750000
1,LAC,8,-3.625000,0.500000,4,4,0,0.500000,226.250000,231.000000,4,3,1,0.500000,-1.937500,6.500000,6,2,0,0.750000,113.687500,111.000000,4,4,0,0.500000,0.125000,1.000000,3,5,0,0.375000,113.000000,116.000000,6,2,0,0.750000,-1.250000,5.000000,5,2,1,0.625000,56.937500,55.000000,1,5,2,0.125000,-0.937500,0.500000,4,4,0,0.500000,56.375000,59.000000,5,3,0,0.625000,0.062500,-4.000000,2,6,0,0.250000,56.625000,58.500000,5,2,1,0.625000,-0.687500,1.500000,4,3,1,0.500000,55.687500,59.000000,5,3,0,0.625000
